In [2]:
import os
import pandas as pd
import re


In [2]:
def dataframes_to_horizontal_text(dataframes, output_path):
    """
    Converts a list of DataFrames with columns @id, @form, @lemma, and @postag to a horizontal 
    text format where each sentence is a separate line, and tokens are separated by spaces. 
    Unicode NO-BREAK SPACE (U+00A0) is replaced by a regular space.

    Parameters:
        dataframes (list of pd.DataFrame): List of DataFrames containing the data.
        output_path (str): Path to save the resulting text file.

    Returns:
        None
    """
    horizontal_output = []
    
    for df in dataframes:
        # Join tokens (@form column) with spaces, replacing NO-BREAK SPACE (U+00A0) with a regular space
        sentence_text = " ".join(token.replace("\u00A0", " ") for token in df["@form"])
        horizontal_output.append(sentence_text)

    # Save to file
    with open(output_path, "w", encoding="utf-8") as f:
        f.write("\n".join(horizontal_output))

In [3]:

def extract_number(file_path):
    # Use regex to find the numeric part of the filename
    return int(re.search(r'(\d+)\.csv$', file_path).group(1))

In [ ]:
#salvo tutte le directory delle opere
lis_dir = sorted(os.listdir('output_gorman_con_relation')) #inserire la directory giusta

In [ ]:
#Preparazione dei file in txt. Da runnare una sola volta

for opera in lis_dir:
    opera_dir = 'output_gorman_con_relation/'+ opera
    all_sorted_csv_filepaths = []
    opera_dir_list = os.listdir(opera_dir) #contiene i file csv di ogni opera i.e. ['0.csv','1.csv']
    #opera_dir.remove('.DS_Store')
    opera_dir_list = sorted(opera_dir_list, key=extract_number) 
    dataframes = [pd.read_csv(opera_dir + '/' + dir, sep=';') for dir in opera_dir_list] #faccio una lista di dataframes (di frasi): Utile per Horizontal
    #all_opera da sistemare in modo da salvare tutte le opere in una lista di dfs
    all_opera = pd.concat(dataframes,ignore_index=True) #concateno tutte le frasi: unico df con frasi di opera (utile per conllu)
    #creo il file orizzontale e lo salvo
    dataframes_to_horizontal_text(dataframes, 'Input Horizontal Treetagger/horizontal_' + opera[:-4]+ '.txt') 

In [3]:
risultati_treetagger_sporchi = os.listdir('risultati_treetagger_sporchi')

In [4]:
risultati_treetagger_sporchi

['horizontal_aeschines-1-1-50-bu1.txt',
 'horizontal_aeschines-1-101-150-bu1.txt',
 'horizontal_aeschines-1-151-196-bu1.txt',
 'horizontal_aeschines-1-51-100-bu1.txt',
 'horizontal_antiphon-1-bu2.txt',
 'horizontal_antiphon-2-bu2.txt',
 'horizontal_antiphon-5-bu2.txt',
 'horizontal_antiphon-6-bu2.txt',
 'horizontal_appian-bc-1-0-1-4-bu1.txt',
 'horizontal_appian-bc-1-11-14-bu1.txt',
 'horizontal_appian-bc-1-5-7-bu1.txt',
 'horizontal_appian-bc-1-8-10-bu1.txt',
 'horizontal_aristotle-politics-book-1-bu1.txt',
 'horizontal_aristotle-politics-book-2-bu2.txt',
 'horizontal_athen12-1-9-2019.txt',
 'horizontal_athen12-10-19-2019.txt',
 'horizontal_athen12-20-29-2019.txt',
 'horizontal_athen12-30-39-jan-15.txt',
 'horizontal_athen12-40-49-jan-15.txt',
 'horizontal_athen12-50-59-jan-15.txt',
 'horizontal_athen12-60-69-jan-15.txt',
 'horizontal_athen12-70-81-jan-15.txt',
 'horizontal_athen13-1-9-jan-15.txt',
 'horizontal_athen13-10-19-jan-15.txt',
 'horizontal_athen13-20-29-jan-15.txt',
 'horiz

In [5]:
dizionario_risultati_treetagger_agdt = {'noun':'n',
                                        'verb':'v',
                                        'preposition': 'r', 
                                        'adjective': 'a', 
                                        'interjection': 'i', 
                                        'proper': 'n', 
                                        'SENT': 'u',
                                          'article': 'l', 
                                          'adverb': 'd', 
                                          'conjunction': 'c',  
                                          'particle': 'g', 
                                          'pronoun': 'p'}
                                        
                

In [6]:

import os
import re

directory = "risultati_treetagger_sporchi/"

tag_line_regex = re.compile(r'^<[^>]*>')

for filename in os.listdir(directory):

    file_path = os.path.join(directory, filename)

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
    except Exception as e:
        print(f"Impossibile leggere il file '{filename}': {e}")
        continue

    kept_lines = []

    for line in lines:
        if tag_line_regex.search(line):
            print(f"File: {filename}\nRiga rimossa: {line}\n")
        else:
            kept_lines.append(line)
    try:
        with open(file_path, "w", encoding="utf-8") as f_out:
            f_out.writelines(kept_lines)
    except Exception as e:
        print(f"Impossibile scrivere sul file '{filename}': {e}")
        continue

In [7]:
lista_risultati_treetagger = []
for risultato in risultati_treetagger_sporchi:
    cols = ['form', 'pos', 'trattino']
    df = pd.read_csv('risultati_treetagger_sporchi/' + risultato, sep = '\t', header = None, names = cols)
    df['pos'] = df['pos'].replace(dizionario_risultati_treetagger_agdt)
    lista_risultati_treetagger.append(df)


In [8]:
opera_names = os.listdir('output_gorman_con_relation')
gorman_dfs = []
for opera in opera_names:
    opera_list = [pd.read_csv('output_gorman_con_relation/' + opera + '/' + frase) for frase in sorted(os.listdir('output_gorman_con_relation/' + opera), key=lambda fn: int(fn.split('.')[0]))]
    opera_finale = pd.concat(opera_list)
    
    opera_finale = opera_finale[['@form', '@postag']].copy()
  
    
    opera_finale['@postag'] = opera_finale['@postag'].str.get(0)
    gorman_dfs.append(opera_finale)

In [9]:
"""
Script per calcolare l'accuracy dei POS tra i risultati di TreeTagger e
quelli dell'edizione Gorman, opera per opera.

Prerequisiti (già prodotti dai tuoi due cicli for):
- tree_dfs: lista di DataFrame (uno per opera) con colonne ['form', 'postag']
           dove 'postag' è stato già normalizzato tramite il dizionario
           (prima lettera del tag, ecc.).
- gorman_dfs: lista di DataFrame (uno per opera) con le stesse colonne e
           lo stesso trattamento sul campo 'postag'.
- opera_names: lista di stringhe con i titoli/identificativi delle opere,
           nello stesso ordine delle due liste di DataFrame.

Il modulo fornisce:
- una funzione `compute_accuracy_per_opera` che restituisce un DataFrame con
  l'accuracy per ogni opera e stampa l'accuracy complessiva.
- una helper `load_dfs` se preferisci caricare i csv da cartelle invece di
  passare direttamente le liste di DataFrame già in memoria.

Usage minimo suggerito:
>>> df_accuracy = compute_accuracy_per_opera(opera_names, tree_dfs, gorman_dfs)
>>> df_accuracy.to_csv('accuracy_per_opera.csv', index=False)
"""

import pandas as pd
from pathlib import Path
from typing import List, Tuple

def load_dfs(dir_path: Path, suffix: str = '.csv') -> Tuple[List[str], List[pd.DataFrame]]:
    """Carica tutti i csv in `dir_path` e restituisce (nomi, DataFrame).
    I file vengono letti in ordine alfabetico per garantire l'allineamento
    con i risultati di TreeTagger.
    """
    file_paths = sorted(dir_path.glob(f'*{suffix}'))
    names = [p.stem for p in file_paths]
    
    return names


def compute_accuracy_per_opera(
    opera_names: List[str],
    tree_dfs: List[pd.DataFrame],
    gorman_dfs: List[pd.DataFrame]
) -> pd.DataFrame:
    """Confronta i POS di TreeTagger e Gorman opera per opera.

    Parameters
    ----------
    opera_names : List[str]
        Titoli o ID delle opere.
    tree_dfs : List[pd.DataFrame]
        DataFrame per TreeTagger (per opera) con colonne ['form', 'postag'].
    gorman_dfs : List[pd.DataFrame]
        DataFrame per Gorman (per opera) con colonne ['form', 'postag'].

    Returns
    -------
    pd.DataFrame
        Tabella con 'opera', 'token_totali', 'corretti' e 'accuracy' (%).
    """

    # Sanity‑check di base
    if not (len(opera_names) == len(tree_dfs) == len(gorman_dfs)):
        raise ValueError(
            "Le liste opera_names, tree_dfs e gorman_dfs devono avere la stessa lunghezza."
        )

    risultati = []
    global_correct = 0
    global_total = 0

    for name, df_tt, df_go in zip(opera_names, tree_dfs, gorman_dfs):
        # Allineamento indici (non si sa mai)
        df_tt = df_tt.reset_index(drop=True)
        df_go = df_go.reset_index(drop=True)

        # Controllo lunghezza
        if len(df_tt) != len(df_go):
            print(f"\nWARNING: Lunghezza disallineata per '{name}': {len(df_tt)} token in TreeTagger vs {len(df_go)} in Gorman.")
            print(f"Continuando l'esecuzione ma i risultati potrebbero essere inaccurati.")
            
            # Trova dove inizia il disallineamento
            min_len = min(len(df_tt), len(df_go))
            mismatch_found = False
            
            # Controlla i token per trovare il punto di disallineamento
            for i in range(min_len):
                if df_tt.iloc[i]['form'] != df_go.iloc[i]['@form']:
                    mismatch_found = True
                    print(f"\nDisallineamento trovato all'indice {i}:")
                    print(f"TreeTagger: {df_tt.iloc[i]['form']}")
                    print(f"Gorman: {df_go.iloc[i]['@form']}")
                    
                    # Mostra alcune righe prima e dopo il disallineamento
                    context_size = 5
                    start_idx = max(0, i - context_size)
                    end_idx = min(min_len, i + context_size + 1)
                    
                    print(f"\nContesto TreeTagger (indici {start_idx}-{end_idx-1}):")
                    print(df_tt.iloc[start_idx:end_idx][['form', 'pos']].to_string())
                    
                    print(f"\nContesto Gorman (indici {start_idx}-{end_idx-1}):")
                    print(df_go.iloc[start_idx:end_idx][['@form', '@postag']].to_string())
                    
                    break
            
            if not mismatch_found:
                print("\nNessun disallineamento trovato nei token comuni. Probabilmente uno dei file ha token aggiuntivi alla fine.")
                
            # Usa la lunghezza minima per il confronto
            df_tt = df_tt.iloc[:min_len].reset_index(drop=True)
            df_go = df_go.iloc[:min_len].reset_index(drop=True)

        # Confronto POS
        correct = (df_tt['pos'] == df_go['@postag']).sum()
        total = len(df_tt)
        acc = round((correct / total) * 100, 2)  # percentuale con 2 decimali

        risultati.append({
            'opera': name,
            'token_totali': total,
            'corretti': correct,
            'accuracy': acc
        })

        global_correct += correct
        global_total += total

    df_risultati = pd.DataFrame(risultati).sort_values('opera').reset_index(drop=True)

    overall_acc = round((global_correct / global_total) * 100, 2) if global_total else None
    print(f"\nAccuracy complessiva sulle opere: {overall_acc}%")

    return df_risultati


In [10]:
opera_names_final = [x[:-4] for x in opera_names]
print(opera_names_final)

['aeschines-1-1-50-bu1', 'aeschines-1-101-150-bu1', 'aeschines-1-151-196-bu1', 'aeschines-1-51-100-bu1', 'antiphon-1-bu2', 'antiphon-2-bu2', 'antiphon-5-bu2', 'antiphon-6-bu2', 'appian-bc-1-0-1-4-bu1', 'appian-bc-1-11-14-bu1', 'appian-bc-1-5-7-bu1', 'appian-bc-1-8-10-bu1', 'aristotle-politics-book-1-bu1', 'aristotle-politics-book-2-bu2', 'athen12-1-9-2019', 'athen12-10-19-2019', 'athen12-20-29-2019', 'athen12-30-39-jan-15', 'athen12-40-49-jan-15', 'athen12-50-59-jan-15', 'athen12-60-69-jan-15', 'athen12-70-81-jan-15', 'athen13-1-9-jan-15', 'athen13-10-19-jan-15', 'athen13-20-29-jan-15', 'athen13-30-39-jan-15', 'athen13-40-49-jan-15', 'athen13-50-59-jan-15', 'athen13-60-69-jan-15', 'athen13-70-79-jan-15', 'athen13-80-89-jan-15', 'athen13-90-95-jan-15', 'dem-59-neaira-2019', 'demosthenes-1-bu1', 'demosthenes-18-1-50-bu2', 'demosthenes-18-101-150-bu2', 'demosthenes-18-151-200-bu2', 'demosthenes-18-201-275-bu1', 'demosthenes-18-276-324-bu1', 'demosthenes-18-51-100-bu1', 'demosthenes-4-phil

In [11]:
import re

regex_number   = r'^(?:[0-9]|1[0-9]|20)$'
# regex per “[0]”–“[20]”
regex_bracketed = r'^\[(?:[0-9]|1[0-9]|20)\]$'

for idx, df in enumerate(gorman_dfs):
    # prendo la colonna @form, tolgo NaN e spazi esterni
    s = df['@form'].fillna('').str.strip()

    # costruisco la maschera di righe DA RIMUOVERE:
    to_remove = (
        s.isin(['[', ']'])            # parentesi isolate
        | s.str.match(regex_number)   # numero “nudo” 0–20
        | s.str.match(regex_bracketed)# “[n]” con n 0–20
    )

    # filtro e riscrivo in dfs
    gorman_dfs[idx] = df[~to_remove]

In [12]:
import re

regex_number   = r'^(?:[0-9]|1[0-9]|20)$'
# regex per “[0]”–“[20]”
regex_bracketed = r'^\[(?:[0-9]|1[0-9]|20)\]$'

for idx, df in enumerate(lista_risultati_treetagger):
    # prendo la colonna @form, tolgo NaN e spazi esterni
    s = df['form'].fillna('').str.strip()

    # costruisco la maschera di righe DA RIMUOVERE:
    to_remove = (
        s.isin(['[', ']'])            # parentesi isolate
        | s.str.match(regex_number)   # numero “nudo” 0–20
        | s.str.match(regex_bracketed)# “[n]” con n 0–20
    )

    # filtro e riscrivo in dfs
    lista_risultati_treetagger[idx] = df[~to_remove]

In [13]:
# Stampa le lunghezze per conferma
print("Lunghezze:")
print(f"opera_names_final: {len(opera_names_final)}")
print(f"lista_risultati_treetagger: {len(lista_risultati_treetagger)}")

# Confronta gli elementi per indice
min_len = min(len(opera_names_final), len(lista_risultati_treetagger))

for i in range(min_len):
    if not isinstance(lista_risultati_treetagger[i], type(gorman_dfs[i])):
        print(f"[Indice {i}] - Possibile mismatch tipo dati")
    # Se hai modo di confrontare contenuti, potresti farlo qui

# Controlla l'elemento extra se c'è davvero una differenza di lunghezza
if len(lista_risultati_treetagger) > len(opera_names_final):
    print("\nElemento extra in lista_risultati_treetagger:")
    extra = lista_risultati_treetagger[len(opera_names_final)]
    print(f"Indice: {len(opera_names_final)}")
    print(f"Contenuto:\n{extra}")
elif len(opera_names_final) > len(lista_risultati_treetagger):
    print("\nElemento extra in opera_names_final:")
    extra = opera_names_final[len(lista_risultati_treetagger)]
    print(f"Indice: {len(lista_risultati_treetagger)}")
    print(f"Contenuto:\n{extra}")
else:
    print("\nNessuna differenza di lunghezza.")


Lunghezze:
opera_names_final: 142
lista_risultati_treetagger: 142

Nessuna differenza di lunghezza.


In [15]:
# lista_risultati_treetagger.pop(5)
# opera_names_final.pop(5)
# gorman_dfs.pop(5)
# lista_risultati_treetagger.pop(6)
# opera_names_final.pop(6)
# gorman_dfs.pop(6)
# # Lista delle opere da rimuovere
# target_names = ['athen12-30-39-jan-15','athen13-60-69-jan-15', 'athen13-30-39-jan-15', 'athen13-40-49-jan-15', 'athen13-80-89-jan-15', 'dem-59-neaira-2019', 'demosthenes-1-bu1', 'demosthenes-18-201-275-bu1', 'demosthenes-18-276-324-bu1', 'demosthenes-46-tree', 'demosthenes-47-tree', 'demosthenes-49-tree', 'demosthenes-50-tree', 'demosthenes-52-tree', 'demosthenes-53-tree','lysias-13-bu1', 'plut-alcib-1-17-bu1', 'plutarch-lycurgus-1-15-bu4', 'plutarch-lycurgus-16-31-bu2','polybius-21-31-47-bu1', 'polybius1-30-39-2017', 'polybius1-40-49-2017', 'polybius1-50-59-2017', 'ps-xen-ath-pol-bu2', 'thuc-1-121-146-bu3', 'thuc-1-41-60-bu3', 'thuc-1-61-80-bu3', 'xen-cyr-7-1-3-tree', 'xen-cyr-7-4-5-tree']

# # Ciclo per rimuoverle da tutte le liste
# for name in target_names:
#     if name in opera_names_final:
#         idx = opera_names_final.index(name)
#         opera_names_final.pop(idx)
#         lista_risultati_treetagger.pop(idx)
#         gorman_dfs.pop(idx)
#         print(f"Rimossa: {name}")
#     else:
#         print(f"{name} non trovato!")


df_accuracy = compute_accuracy_per_opera(opera_names_final, lista_risultati_treetagger, gorman_dfs)

#df_accuracy.to_csv('accuracy_per_opera_sporchi.csv', index=False)




Continuando l'esecuzione ma i risultati potrebbero essere inaccurati.

Disallineamento trovato all'indice 160:
TreeTagger: :
Gorman: <

Contesto TreeTagger (indici 155-165):
          form pos
155         ὡς   c
156  ἀπέκτεινε   v
157        τὸν   l
158      ἄνδρα   n
159          .   u
160          :   v
161     οὐδεὶς   p
162        γὰρ   g
163         ἂν   d
164        τὸν   l
165    ἔσχατον   a

Contesto Gorman (indici 155-165):
          @form @postag
155          ὡς       c
156   ἀπέκτεινε       v
157         τὸν       l
158       ἄνδρα       n
159           .       u
160           <       u
161          Οὔ       d
162         -τε       d
163         γὰρ       d
164  κακούργους       a
165       εἰκὸς       v

Continuando l'esecuzione ma i risultati potrebbero essere inaccurati.

Disallineamento trovato all'indice 25:
TreeTagger: ἄρα
Gorman: <

Contesto TreeTagger (indici 20-30):
           form pos
20          τις   p
21        ταῦτα   p
22      εὔξαιτο   v
23            :   v
